## Comencemos importando las bibliotecas que usaremos:

In [1]:
# !pip install lxml

In [64]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup as bs
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [65]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Paris\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Paris\AppData\Local\Temp\ipykernel_5576\794677937.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://exoplanets.nasa.gov/discovery/exoplanet-catalog/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)
time.sleep(5)

In [5]:
elements_50 = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/div[3]/section[2]/div/section[2]/div/div/article/div/div[2]/div[1]/div[2]/div[2]/div/select/option[3]")
elements_50.click()
time.sleep(2)

C:\Users\Paris\AppData\Local\Temp\ipykernel_11644\1657767747.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  elements_50 = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/div[3]/section[2]/div/section[2]/div/div/article/div/div[2]/div[1]/div[2]/div[2]/div/select/option[3]")


# Web Scraping 

In [6]:
masses = []
distances = []
years = []
names = []
stellars = []
planet_links = []

for i in range(int(np.ceil(5030/50))):
	soup = bs(driver.page_source,"lxml")
	exo_names = soup.select("div.datasearch ul.exoplanet li.display_name")
	exo_years = soup.select("div.datasearch ul.exoplanet li.discovery_date")
	exo_distances = soup.select("div.datasearch ul.exoplanet li.st_dist")
	exo_masses = soup.select("div.datasearch ul.exoplanet li.mass_display")
	exo_stellars = soup.select("div.datasearch ul.exoplanet li.st_optmag")
	links = soup.select("ul.exoplanet li.display_name a")

	for name in exo_names: 
		names.append(name.get_text())
	for year in exo_years: 
		years.append(year.get_text())
	for distance in exo_distances: 
		distances.append(distance.get_text())
	for mass in exo_masses: 
		masses.append(mass.get_text())
	for stellar in exo_stellars: 
		stellars.append(stellar.get_text())
	for link in links:
		planet_links.append('https://exoplanets.nasa.gov' + link.attrs['href'])
		

	loadMore = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/div[3]/section[2]/div/section[2]/div/div/article/div/div[2]/footer/div/div/div/nav/span[2]/a")
	loadMore.click()
	time.sleep(2)

C:\Users\Paris\AppData\Local\Temp\ipykernel_11644\3382208279.py:31: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  loadMore = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/div[3]/section[2]/div/section[2]/div/div/article/div/div[2]/footer/div/div/div/nav/span[2]/a")


In [7]:
#Create & Save Dataframe

In [8]:
exo_links = pd.DataFrame({'links':planet_links})
exo_links.to_csv("data/raw/exo_links.csv")

In [9]:
data = { "names": names,
		"mass": masses,
		"distance": distances,
		"discovery_year": years,
		"stellar_magnitude": stellars
		}
df = pd.DataFrame(data)
df.to_csv("data/raw/test.csv")

# Web Scraping Exoplanets

In [75]:
# names = []
# date = []
type = []
radius = []
orbital_radius = []
orbital_days = []
detection_method = []
eccentricity = []

In [76]:
i=0
for link in planet_links[len(type):]:
	driver.get(link)
	time.sleep(2)
	soup = bs(driver.page_source,"lxml")
	exo_data = soup.select("table.information_grid div.value")
	# name = soup.select("h1.article_title")
	# names.append(name[0].get_text().split('\n')[1])
	type.append(exo_data[0].get_text().split('\n')[1])
	radius.append(exo_data[3].get_text().split('\n')[1])
	orbital_radius.append(exo_data[4].get_text().split('\n')[1].split(' ')[0])
	orbital_days.append(exo_data[5].get_text().split('\n')[1].split(' ')[0])
	eccentricity.append(exo_data[6].get_text().split('\n')[1])
	detection_method.append(exo_data[7].get_text().split('\n')[1])
	i += 1
	if i%100 == 0:
		print(i)

100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 

In [ ]:
#Save data on csv

In [77]:
nasa_catalog = pd.DataFrame({'planet_type':type,
							'radius':radius,
							'orbital_radius':orbital_radius,
							'orbital_days':orbital_days,
							'eccentricity':eccentricity,
							'detection_method':detection_method})

In [78]:
nasa_catalog.to_csv("data/raw/test_2.csv")

In [ ]:
#Load data from csv

In [33]:
nasa_catalog = pd.read_csv("data/raw/test_2.csv", index_col=0)

In [10]:
type = list(nasa_catalog.planet_type)
radius = list(nasa_catalog.radius)
orbital_radius = list(nasa_catalog.orbital_radius)
orbital_days = list(nasa_catalog.orbital_days)
detection_method = list(nasa_catalog.detection_method)
eccentricity = list(nasa_catalog.eccentricity)

In [72]:
#Load planet_links
df_planets = pd.read_csv("data/raw/exo_links.csv", index_col=0)
planet_links = list(df_planets.links)

In [ ]:
df_planet_links = pd.read_csv("data/raw/exo_links.csv", index_col=0)
planet_links = list(df_planet_links.links)

In [4]:
df_1 = pd.read_csv("data/raw/exo_links.csv", index_col=0)
df_2 = pd.read_csv("data/raw/exo_links_old.csv", index_col=0)

In [5]:
a = df_1.links.values

In [6]:
b = df_2.links.values

In [12]:
planet_links = list(set(a) - set(b))

In [48]:
nasa_catalog.Date = nasa_catalog.Date.astype(int)

In [51]:
nasa_catalog[nasa_catalog.Date == 2022]

,planet_type,radius,orbital_radius,orbital_days,eccentricity,detection_method,Date
11,Neptune-like,0.332 x Jupiter(estimate),Unknown,124.6,0.22,Radial Velocity,2022
23,Gas Giant,1.2 x Jupiter(estimate),Unknown,43,0.0,Eclipse Timing Variations,2022
34,Gas Giant,1.3 x Jupiter,Unknown,2.1,0.0,Transit,2022
84,Super Earth,1.83 x Earth(estimate),0.05366,10.8,0.33,Radial Velocity,2022
97,Neptune-like,1.85 x Earth,0.0182,1,0.13,Transit,2022
131,Gas Giant,1.44 x Jupiter,Unknown,1.5,0.0,Transit,2022
140,Neptune-like,0.215 x Jupiter(estimate),1.8,7.6,0.0,Gravitational Microlensing,2022
149,Gas Giant,1.322 x Jupiter,0.0467,3.7,0.0,Transit,2022
153,Gas Giant,1.36 x Jupiter,Unknown,2.1,0.0,Transit,2022
156,Gas Giant,1.3 x Jupiter,0.0561,4.1,0.03,Transit,2022


In [61]:
name = soup.select("h1.article_title")
names.append(name[0].get_text().split('\n')[1])

'OGLE-2018-BLG-0596L b'